In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile

In [ ]:
zf = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/train.csv.zip') 
train = pd.read_csv(zf.open('train.csv'))
zf = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/test.csv.zip') 
test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/macro.csv.zip') 
macro = pd.read_csv(zf.open('macro.csv'))
zf = zipfile.ZipFile('/kaggle/input/sberbank-russian-housing-market/sample_submission.csv.zip') 
submission = pd.read_csv(zf.open('sample_submission.csv'))

In [ ]:
print(f'{train.shape} - Размерность обучающей')
print(f'{test.shape} - Размерность тестовой')

# Функция создания дф с пропусками:


In [ ]:
def missing_values(df):
    mis_val = df.isnull().sum()
    mis_val_per = df.isnull().sum()/df.shape[0]

    res_df = pd.concat([mis_val, mis_val_per], axis=1)
    res_df = res_df.rename(columns={0:'Mis. values', 1:'% Mis. values'})
    res_df = res_df.sort_values(by='% Mis. values', ascending=False)
    
    res_df = res_df[res_df['Mis. values'] > 0]
    return res_df

# Рассмотрим цель

In [ ]:
train['price_doc'].describe()

In [ ]:
train.info()

In [ ]:
plt.hist(train['price_doc'], bins = 40, edgecolor = 'black')

plt.xlabel('price')
plt.ylabel('quantity')
plt.show()

In [ ]:
plt.figure(figsize=(18, 15))
sns.displot(train['price_doc'], bins=100, kde=True)
plt.title('price_doc', fontsize=18)
plt.xlabel('price_doc', fontsize=12)
plt.show()

видно, что точки распределены большей частью в (0.0;0.2) и имеется выступ >1.0 (*10^8)

In [ ]:
train['price_doc_log'] = np.log(train['price_doc'])

в задании логарифмическая оценка, поэтому следует создать логарифм цели для дальнейшей оценки

In [ ]:
train.head()

In [ ]:
#first_quantile = train['price_doc'].describe()['25%']
#third_quantile = train['price_doc'].describe()['75%']

#iqr = third_quantile - first_quantile


#train = train[(train['price_doc'] > (first_quantile - 3 * iqr)) &
#            (train['price_doc'] < (third_quantile + 3 * iqr))]
#plt.hist(train['price_doc'], bins=50, edgecolor = 'black')
#plt.xlabel('price * 10^7')
#plt.ylabel('quantity')
#plt.show()

In [ ]:
mis_cols = missing_values(train)
mis_cols_i = list(mis_cols.index)
len(mis_cols_i)

всего имеется 51 признак, в которых присутствуют пропуски

In [ ]:
plt.figure(figsize=(28, 20))
sns.heatmap(train[mis_cols_i + ['price_doc']].corr(), annot=True)
plt.show()

В последней строке указаны зависимости признаков и цели

In [ ]:
plt.figure(figsize=(12, 10))
plt.subplot(2, 2, 1)
plt.hist(train['build_year'])
plt.xlabel('Build_year')
plt.ylabel('Quantity')
plt.subplot(2, 2, 2)
plt.hist(train['life_sq'])
plt.xlabel('life_sq')
plt.ylabel('Quantity')
plt.subplot(2, 2, 3)
plt.hist(train['build_count_brick'])
plt.xlabel('build_count_brick')
plt.ylabel('Quantity')
plt.subplot(2, 2, 4)
plt.hist(train['state'])
plt.xlabel('state')
plt.ylabel('Quantity')
plt.show()

Мое предположение, что из-за бОльших значений распределение становится таким. 

Меня заинтересовали такие признаки, как num_room, life_sq, build_count_bvrick, state.

In [ ]:
train['build_year'].value_counts() # Уже можно заметить, что есть такие цифры как 71 0 4965

In [ ]:
train['build_year'] = train['build_year'].replace({20052009: 2007, 0: 2000})
train['build_year'] = train['build_year'].replace({4965: train['build_year'].mode().iloc[0]})
train['build_year'] = train['build_year'].replace({1: 2001, 2: 2002, 3: 2003})
train['build_year'] = train['build_year'].replace({71: 1971, 20: 1920, 215: 2015})

заменим ошибочные значения

In [ ]:
train[(train['build_year'] > 2021) | (train['build_year'] < 1000) ]['build_year'].sort_values(ascending=False).value_counts()

Проверим остались ли "необычные" значения

In [ ]:
train['build_year'].mode()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(train['build_year'], bins=50)
plt.show()

Посмотрим распределение build_year после внесенных изменений.

In [ ]:
train.corr()['price_doc']['build_year']

In [ ]:
plt.scatter(y='price_doc', x='life_sq', data=train) #заметно необычное значение > 7000
plt.ylabel('price_doc')
plt.xlabel('life_sq')

Оценим зависимость цены от жилой площади. Заметим, что есть обособленное значение >7000, при котором цена очень мала. Подозрительно!

In [ ]:
plt.hist(train['life_sq'])
plt.show()

По всей видимости данное значение влияет на вид распределения

In [ ]:
train = train[train['life_sq']<7000]

In [ ]:
first_quantile = train['life_sq'].describe()['25%']
third_quantile = train['life_sq'].describe()['75%']

iqr = third_quantile - first_quantile


train = train[(train['life_sq'] > (first_quantile - 3 * iqr)) &
            (train['life_sq'] < (third_quantile + 3 * iqr))]
plt.hist(train['life_sq'], edgecolor = 'black')
plt.xlabel('life_sq')
plt.ylabel('quantity')
plt.show()

Приведем признак к нормальному распределению, используя интерквартильный размах

In [ ]:
train.shape

In [ ]:
plt.scatter(y='price_doc', x='build_count_brick', data=train) #есть выраженное накопление точек до кол-ва 400, далее отчужденные значения

Нельзя сделать никаких выводов, потому что при разных количествах цена на одном уровне

In [ ]:
plt.scatter(y='price_doc', x='num_room', data=train) # из-за пропусков получился рассеянный график, у которого есть необычные значения >=10

Интересно, почему же аномальное количество комнат (17 и 19) не влияет должным образом на цену. Быть может это ошибка?!

In [ ]:
train = train.drop(index=[11621, 26713])

Продолжим без них

In [ ]:
plt.hist(train['num_room'])

In [ ]:
plt.hist(train[train['num_room']<10]['num_room'])

In [ ]:
plt.hist(train[['num_room']])

Теперь распределение начинает напоминать нормальное

In [ ]:
# переменные с подсчетом чего-либо floor, max_floor, material, все count'ы можно заполнить модой
#mis_col = list(missing_values(train).index)
#mode_cols = [col for col in mis_col if (col=='floor') or (col=='max_floor') or ('count' in col)]
#misses_names = list(mis_cols.index)
#on_mode = []
#for col in misses_names:
#    if 'count' in col or col=='floor' or col=='build_year' or col=='material' or col=='num_room' \
#    or col=='green_part_2000' or col=='ID_railroad_station_walk' or col=='state' or col =='hospital_beds_raion'\
#    or 'quota' in col:
#        on_mode.append(col)
#on_mode
cols_fillna_mode = ['floor',
 'product_type', 
 'build_year',                 
 'max_floor',
 'material',
 'num_room',
 'state',
 'preschool_quota',
 'school_quota',
 'hospital_beds_raion',
 'raion_build_count_with_material_info',
 'raion_build_count_with_builddate_info',
 'ID_railroad_station_walk',
 'build_count_block',
 'build_count_wood',
 'build_count_frame',
 'build_count_brick',
 'build_count_monolith',
 'build_count_panel',
 'build_count_foam',
 'build_count_slag',
 'build_count_mix',
 'build_count_before_1920',
 'build_count_1921-1945',
 'build_count_1946-1970',
 'build_count_1971-1995',
 'build_count_after_1995',
 'green_part_2000']

cols_fillna_mean = ['life_sq',
 'kitch_sq',
 'metro_min_walk',
 'metro_km_walk',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'cafe_sum_500_min_price_avg',
 'cafe_sum_500_max_price_avg',
 'cafe_avg_price_500',
 'cafe_sum_1000_min_price_avg',
 'cafe_sum_1000_max_price_avg',
 'cafe_avg_price_1000',
 'cafe_sum_1500_min_price_avg',
 'cafe_sum_1500_max_price_avg',
 'cafe_avg_price_1500',
 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000',
 'cafe_sum_3000_min_price_avg',
 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000',
 'prom_part_5000',
 'cafe_sum_5000_min_price_avg',
 'cafe_sum_5000_max_price_avg',
 'cafe_avg_price_5000']

Составим списки признаков, в которых требуется проработать пропуски. Соответсвенно для категориальных и num признаков.

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean').fit(train[cols_fillna_mean])
train[cols_fillna_mean]=imputer.transform(train[cols_fillna_mean])

test[cols_fillna_mean]=imputer.transform(test[cols_fillna_mean])

Заполним num признаки средним значением

In [ ]:
imputer=SimpleImputer(strategy='most_frequent').fit(train[cols_fillna_mode])
train[cols_fillna_mode]=imputer.transform(train[cols_fillna_mode])

test[cols_fillna_mode]=imputer.transform(test[cols_fillna_mode])

Заполним категориальные модой

In [ ]:
print(train.isnull().sum().sum())
print(test.isnull().sum().sum())

In [ ]:
num_feat = list(train.dtypes[train.dtypes != "object"].index)
cat_feat = list(train.dtypes[train.dtypes == "object"].index)


Составим списки соответствующих признаков

In [ ]:
for col in cat_feat:
    if 'floor' in str(col) or 'count' in str(col) or \
    'build' in str(col) or col in ['state', 'material', 'preschool_quota', 'school_quota', 'green_part_2000', 'ID_railroad_station_walk', 'hospital_beds_raion', \
                                  'num_room']:
        num_feat.append(col)
        cat_feat.remove(col)

Перенесем некоторые прищнаки

In [ ]:
len(num_feat)

In [ ]:
train.shape

In [ ]:
for cat in cat_feat:
    print(cat, train[cat].nunique())

Оценим категориальные признаки, можно заметить, что многие являются бинарными.

In [ ]:
for feat in cat_feat:
    print(feat)
    if feat not in ['product_type', 'ecology', 'sub_area']:
        print(train[feat].unique())

В таких признаках значения no, yes.

In [ ]:
sns.stripplot(y=train['price_doc'], x=train['ecology'])

В основном случае значение экологии не влияет на цену

In [ ]:
sns.stripplot(y=train['price_doc'], x=train['product_type'])

В среднем случае Investment преобладает над OwnerCcupier.


In [ ]:
plt.figure(figsize=(25, 10))
plt.xticks(rotation=45)
sns.stripplot(y=train['price_doc'], x=train['sub_area'])

В костино-люхтомском увлеиченное значение price в среднем. также в Тропарево-Никулино, Гагинском.

In [ ]:
for_graph = cat_feat.remove('timestamp')

In [ ]:
rows = 5
cols = 3

fig, axs = plt.subplots(rows, cols, figsize=(cols*5,rows*3))


for r in range(0, rows):
    for c in range(0, cols):
        i = r*cols + c
        i_col = cat_feat[i]
        sns.countplot(x=i_col, data=train, ax=axs[r][c])

In [ ]:
fig, ax = plt.subplots(figsize=(10,30))
sns.barplot(y='sub_area', x='price_doc', data=train, orient='h')
plt.title('sub-area', fontsize=18)
plt.show()

Стоит обратить внимание на Арбат и Хамовники!

In [ ]:
train[train['full_sq']<train['life_sq']]

In [ ]:
train = train.drop(train[train['full_sq']<train['life_sq']].index)

Очень сильно настораживают то, что есть объекты, у которых  жилая площадь больше общей. Нелогично!

In [ ]:
train.shape

In [ ]:
cat_feat

In [ ]:
cat_feat.remove('timestamp')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='error')

encode_train = pd.DataFrame(encoder.fit_transform(train[cat_feat]).toarray())
encode_test = pd.DataFrame(encoder.transform(test[cat_feat]).toarray())
encode_train.columns = encoder.get_feature_names(cat_feat)
encode_test.columns = encoder.get_feature_names(cat_feat)

encode_train.index = train.index
encode_test.index = test.index

num_train = train.drop(cat_feat, axis=1)
num_test = test.drop(cat_feat, axis=1)

train = pd.concat([num_train, encode_train], axis=1)
test = pd.concat([num_test, encode_test], axis=1)

Сделаем onehotencoder для кат. признаков. Назовем столбцы соответствующим образом. Перенесем в преобразованный дф нужные индексы.
В исходном оставим только num для того, чтобы далее склеить Onehotencoded столбцы и num.

In [ ]:
train[(train['num_room']>=0) & (train['life_sq']==0)]

В 45 объектах указано количество комнат, но при этом не указано сколько жилой площади. Разве так делается?

In [ ]:
fu = list(train[(train['num_room']>=0) & (train['life_sq']==0)].index)
train = train.drop(index=fu)

In [ ]:
#test = test.drop(index=list(test[test['life_sq']==0].index))

In [ ]:
#train['room_size'] = train['life_sq']/train['num_room']
zeros = list(train[train['num_room']==0].index)

у 4 объектов вообще нет комнат! а на сколько я помню здесь пристутствуют торлько apartments!

In [ ]:
train = train.drop(index=zeros)

При наличии жилой площади и кол-ва комнат само собой напрашивается вывести средний размер комнаты.

In [ ]:
train['room_size'] = train['life_sq']/train['num_room']
test['room_size'] = test['life_sq']/test['num_room']

In [ ]:
set(train.columns) - set(test.columns)

In [ ]:
train.head()

In [ ]:
#macro.head()
on_merge = macro[['timestamp', 'oil_urals', 'usdrub', 'eurrub', 'brent']]

Выберем признаки с наименьшим количеством пропусков из macro

In [ ]:
on_merge = on_merge.drop(index=[0,1,2])

In [ ]:
train['timestamp']

In [ ]:
train = train.merge(on_merge, on='timestamp')
test = test.merge(on_merge, on='timestamp')

введем такие столбцы в наши обучающие и тестовые выборки!

In [ ]:
train[['timestamp', 'price_doc']]

Следует что-то сделать и с датами!

In [ ]:
train['month'] = train['timestamp'].apply(lambda x: x[5:7]).astype(int)

In [ ]:
test['month'] = test['timestamp'].apply(lambda x: x[5:7]).astype(int)

In [ ]:
train['year'] = train['timestamp'].apply(lambda x: x[:4]).astype(int)
test['year'] = test['timestamp'].apply(lambda x: x[:4]).astype(int)

In [ ]:
train.head()

In [ ]:
sns.barplot(x='year', y='price_doc', data=train)
plt.show()

каждый год происходит увеличение стоимости жилья

In [ ]:
train.corr()['price_doc_log']['year']

In [ ]:
#train['year'] = train['timestamp'].apply(lambda x: x[:4]).astype(int)
#test['year'] = test['timestamp'].apply(lambda x: x[:4]).astype(int)

In [ ]:
#sns.barplot(x='year', y='price_doc', data=train)
#plt.show()

In [ ]:
sns.barplot(x='month', y='price_doc', data=train)
plt.show()

В сентябре и трех месяцев весны наблюдается среднее повышение стоимости жилья

In [ ]:
train['is_interesting'] = train['timestamp'].apply(lambda x: 1.0 if str(x[5:7]) in ['03', '04', '05'] else 0.0)
test['is_interesting'] = test['timestamp'].apply(lambda x: 1.0 if str(x[5:7]) in ['03', '04', '05'] else 0.0)

In [ ]:
plt.figure(figsize=(12, 8))
plt.xticks(rotation=45)
t = train.groupby(by=['floor'])['price_doc'].aggregate(np.median).reset_index()
sns.pointplot(x='floor', y='price_doc', data=t)

наблюдается тенденция на увеличение стоимости при возрастании этажа, но в абсциссе 33 какое-то необычное значение!

In [ ]:
train[train['floor'] == 33]

Ага! Здесь у нас максимальное кол-во этажей 9, а имущество на 33 этаже! Такое допускать нельзя!

In [ ]:
train = train.drop(index=train[train['floor'] == 33].index)

In [ ]:
sns.barplot(x='is_interesting', y='price_doc', data=train)

Думаю сразу можно выразить год и месяц

In [ ]:
train['year_month'] = train['timestamp'].apply(lambda x: x[:4] + x[5:7]).astype(int)
test['year_month'] = test['timestamp'].apply(lambda x: x[:4] + x[5:7]).astype(int)

In [ ]:
train.groupby(train['num_room'])['price_doc'].aggregate(np.median).reset_index()

In [ ]:
plt.figure(figsize=(12, 8))
plt.xticks(rotation=45)
t = train.groupby(by=['num_room'])['price_doc'].aggregate(np.median).reset_index()
sns.pointplot(x='num_room', y='price_doc', data=t)

In [ ]:
train[train['num_room']==10]

In [ ]:
#%pip install fastcluster
plt.figure(figsize=(20, 10))
sns.clustermap(train.corr())
plt.show()

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
train.describe()

имеются признаки разных порядков. нехорошо! 


In [ ]:
y = train['price_doc_log']
X = train.drop(['price_doc_log'], axis=1)


In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
X = X.drop('timestamp', axis=1)
X = X.drop('price_doc', axis=1)
X = X.drop('id', axis=1)

In [ ]:
#X = X.drop('timestamp', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_valid_scaled = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lin_predictions_log = lr_model.predict(X_valid_scaled)
lin_predictions = np.exp(lin_predictions_log)

In [ ]:
np.any(np.isfinite(lin_predictions))

In [ ]:
from sklearn.metrics import mean_squared_log_error
print('RMSLE linear regression:', np.sqrt(mean_squared_log_error(np.exp(y_valid), lin_predictions)))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
#from sklearn.model_selection import GridSearchCV
#model = DecisionTreeRegressor()
#params = {
#    "criterion": ["mse", "mae"],
#              "min_samples_split": [10, 20],
#              "max_depth": [2, 6, 7, 8]
#}
#model.fit(X_train_scaled, y_train)
#clf = GridSearchCV(estimator=model, param_grid=params, cv=5)
#clf.fit(X_train_scaled, y_train)
#clf.best_params_

In [ ]:
  
tree = DecisionTreeRegressor(random_state = 42, max_depth=7, min_samples_split=20)  
tree.fit(X_train_scaled, y_train)

tree_predictions_log = tree.predict(X_valid_scaled)
tree_predictions = np.exp(tree_predictions_log)

In [ ]:
print('RMSLE Decision Tree:', np.sqrt(mean_squared_log_error(np.exp(y_valid), tree_predictions)))

In [ ]:
import xgboost

In [ ]:
xg_train = xgboost.DMatrix(X_train_scaled, y_train)
xg_valid = xgboost.DMatrix(X_valid_scaled, y_valid)

In [ ]:
xgb = xgboost.XGBRegressor(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
#params = {'objective':['reg:squarederror'],
#              'learning_rate': [0.03, 0.05, 0.07],
#              'max_depth': [5, 7],
#              'min_child_weight': [4],
#              'subsample': [0.7],
#              'colsample_bytree': [0.7],
#              'n_estimators': [500]}
#xgb_grid = GridSearchCV(xgb,
#                        params,
#                        cv = 2,
#                        n_jobs = -1,
#                        verbose=True)

In [ ]:
#xgb_grid.fit(X_train_scaled, y_train)
#print(xgb_grid.best_score_)
#print(xgb_grid.best_params_)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.03, max_delta_step=0, max_depth=5,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
#xgbg = xgb_grid.best_estimator_
xgbg = xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1, importance_type='gain', interaction_constraints='', learning_rate=0.03, max_delta_step=0, max_depth=5, min_child_weight=4, monotone_constraints='()', n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, subsample=0.7, tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
xgbg.fit(X_train_scaled, y_train)

In [ ]:
xgb_predictions_log = xgbg.predict(X_valid_scaled)

In [ ]:
xgb_predictions = np.exp(xgb_predictions_log)

In [ ]:
print('RMSLE XGB:', np.sqrt(mean_squared_log_error(np.exp(y_valid), xgb_predictions)))

In [ ]:
#tree.feature_importances_
plt.figure(figsize=(28, 15))
fig, ax = plt.subplots()
importances = tree.feature_importances_

feature_names = [i for i in X.columns]
importances = pd.Series(importances, index=feature_names)
importances.plot.bar(ax=ax)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 10))
_= xgboost.plot_importance(xgbg, max_num_features=30, height=0.5, ax=ax)

In [ ]:
#from sklearn.ensemble import AdaBoostRegressor
#from sklearn.model_selection import GridSearchCV
#ada = AdaBoostRegressor()
#model = DecisionTreeRegressor()
#params = {
#    'n_estimators': [10, 20, 30, 40, 50],
#    'learning_rate': [0.03, 0.05, 0.07],
#    'loss': ['linear', 'square', 'exponential']
#}
#model.fit(X_train_scaled, y_train)
#reg = GridSearchCV(estimator=ada, param_grid=params, cv=5)
#reg.fit(X_train_scaled, y_train)
#reg.best_params_

#print('RMSLE XGB (valid dataset):', np.sqrt(mean_squared_log_error(np.exp(y_valid), preds)))

In [ ]:
test = test.drop('timestamp', axis=1)

In [ ]:
test = test.drop('id', axis=1)

In [ ]:
set(X.columns) - set(test.columns)

In [ ]:
df_test_scaled = pd.DataFrame(scaler.transform(test), columns=test.columns)

In [ ]:
df_test_scaled

In [ ]:
ylog_pred = xgbg.predict(df_test_scaled)
y_pred = np.exp(ylog_pred)
id_test = test.index + 30474
df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})
#df_sub
df_sub.to_csv('sub.csv', index=False)

In [ ]:
df_sub

In [ ]:
submission

In [ ]:
train.head()

In [ ]:
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
create_download_link(df_sub)